# Ray RLlib Training for Zelda Oracle of Seasons

This notebook deploys a Ray cluster on OpenShift/Kubernetes and submits a distributed training job.

Based on the Double Dragon KubeRay implementation.


## ⚠️ RBAC Setup Required

**Before running this notebook**, you need to grant RBAC permissions to the service account.

Run these commands in a terminal:

```bash
cd /Users/cnuland/hello-chris-rl-llm-zelda

# Apply RBAC permissions
oc apply -f ops/openshift/rbac.yaml

# Verify permissions
oc auth can-i list rayclusters --as=system:serviceaccount:zelda-hybrid-rl-llm:zelda-rl-training -n zelda-hybrid-rl-llm
oc auth can-i create rayclusters --as=system:serviceaccount:zelda-hybrid-rl-llm:zelda-rl-training -n zelda-hybrid-rl-llm
```

All should return `yes` ✅

**Then proceed with the cells below.**


In [ ]:
!pip install codeflare-sdk


In [ ]:
# Updated imports for newer codeflare_sdk versions
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration
from codeflare_sdk.cluster.auth import TokenAuthentication
import os
import time


In [ ]:
# Authenticate with OpenShift
# Get your token: oc whoami -t
# Get your server: oc cluster-info

auth = TokenAuthentication(
    token = 'YOUR_TOKEN_HERE',  # Replace with: oc whoami -t
    server = 'YOUR_SERVER_HERE',  # Replace with: oc cluster-info
    skip_tls=False
)
auth.login()


In [ ]:
# First, let's check what parameters ClusterConfiguration actually accepts
import inspect
sig = inspect.signature(ClusterConfiguration.__init__)
print("Available ClusterConfiguration parameters:")
for param_name, param in sig.parameters.items():
    if param_name != 'self':
        default = param.default if param.default != inspect.Parameter.empty else "REQUIRED"
        print(f"  {param_name}: {default}")


In [ ]:
# Configure Ray Cluster
# ✅ REUSING the Double Dragon image - it already has everything we need!
# 🎮 PyBoy emulator works great on CPU - no GPUs needed!
# ✅ Ultra-minimal resources to guarantee scheduling on available nodes!

cluster = Cluster(ClusterConfiguration(
    name='zelda-rl',
    namespace='zelda-hybrid-rl-llm',
    num_workers=2,                    # ✅ Reduce from 3 to 2
    worker_cpu_requests=1,            # ✅ Reduce from 8 to 1  
    worker_cpu_limits=2,              # ✅ Reduce from higher to 2
    worker_memory_requests="2Gi",     # ✅ Reduce from 12Gi to 2Gi
    worker_memory_limits="4Gi",       # ✅ Reduce from 16Gi to 4Gi
    head_cpu_requests=1,              # ✅ Small head node
    head_cpu_limits=2,
    head_memory_requests="2Gi",
    head_memory_limits="4Gi",
    
    # No GPUs needed - PyBoy runs great on CPU!
    num_gpus=0,
    
    # ✅ Using your existing DD image
    image="quay.io/cnuland/dd-kuberay-worker:latest",
))

print(f"✅ Cluster configuration created:")
print(f"   Name: {cluster.config.name}")
print(f"   Namespace: {cluster.config.namespace}")
print(f"   Workers: {cluster.config.num_workers}")
print(f"   Worker CPUs: 1 request, 2 limit")
print(f"   Worker Memory: 2-4 GB")
print(f"   Head CPUs: 1 request, 2 limit")
print(f"   Head Memory: 2-4 GB")
print(f"   Image: {cluster.config.image}")
print(f"   Total pods: 3 (1 head + 2 workers)")
print(f"   Total resources: 3-6 CPUs, 6-12 GB RAM")
print(f"   Parallel games: 6 (3 per worker)")
print(f"\n✅ Ultra-minimal resources - should fit on any available nodes!")
print(f"✅ Kueue queues (zelda-ray-queue → ray-cluster-queue) are ready!")


In [ ]:
# Create the Ray cluster
print("Creating Ray cluster...")
cluster.up()

# Check status immediately
print("\nChecking cluster status...")
status_info = cluster.status()
print(f"Status: {status_info}")

# Try to wait for ready (with better error handling)
try:
    print("\nWaiting for cluster to be ready...")
    cluster.wait_ready(timeout=600)  # 10 minute timeout
    print("✅ Cluster is ready!")
except Exception as e:
    print(f"⚠️ Error waiting for cluster: {e}")
    print("\nCluster details:")
    print(cluster.details())


In [ ]:
# Diagnostic: Check cluster status and image pull issues
import subprocess

print("🔍 DIAGNOSTIC: Checking Ray cluster deployment...\n")

try:
    # Check RayCluster resource
    print("1️⃣ RayCluster resource:")
    result = subprocess.run(
        ["oc", "get", "raycluster", "-n", "zelda-hybrid-rl-llm"],
        capture_output=True, text=True, timeout=10
    )
    print(result.stdout if result.returncode == 0 else result.stderr)
    
    # Check pods
    print("\n2️⃣ Pods in namespace:")
    result2 = subprocess.run(
        ["oc", "get", "pods", "-n", "zelda-hybrid-rl-llm"],
        capture_output=True, text=True, timeout=10
    )
    print(result2.stdout if result2.returncode == 0 else result2.stderr)
    
    # Check for image pull errors in events
    print("\n3️⃣ Recent events (looking for ImagePullBackOff errors):")
    result3 = subprocess.run(
        ["oc", "get", "events", "-n", "zelda-hybrid-rl-llm", 
         "--sort-by=.lastTimestamp", "--field-selector=type=Warning"],
        capture_output=True, text=True, timeout=10
    )
    events = result3.stdout if result3.returncode == 0 else result3.stderr
    print(events if events.strip() else "No warning events found")
    
    # Check image pull secrets
    print("\n4️⃣ Image pull secrets in namespace:")
    result4 = subprocess.run(
        ["oc", "get", "secrets", "-n", "zelda-hybrid-rl-llm", 
         "-o", "jsonpath={.items[?(@.type==\"kubernetes.io/dockerconfigjson\")].metadata.name}"],
        capture_output=True, text=True, timeout=10
    )
    secrets = result4.stdout if result4.returncode == 0 else result4.stderr
    print(secrets if secrets.strip() else "⚠️  No dockerconfigjson secrets found!")
    
except Exception as e:
    print(f"❌ Error running diagnostic commands: {e}")

print("\n" + "="*60)
print("💡 IMAGE PULL FIX:")
print("="*60)
print("If you see 'ImagePullBackOff' errors, you need to:")
print("1. Check if quay.io/cnuland/dd-kuberay-worker:latest exists")
print("2. If private, create image pull secret:")
print("   oc create secret docker-registry quay-pull-secret \\")
print("     --docker-server=quay.io \\")
print("     --docker-username=YOUR_USERNAME \\")
print("     --docker-password=YOUR_PASSWORD \\")
print("     -n zelda-hybrid-rl-llm")
print("3. Then add to ClusterConfiguration:")
print("   image_pull_secrets=['quay-pull-secret']")


In [ ]:
# Get cluster details and submit job
clusterDetails = cluster.details()
print(f"Ray Dashboard URL: {clusterDetails.dashboard}")

client = cluster.job_client

# Configure environment variables
env_vars = {
    # S3 Storage (MinIO)
    'S3_ACCESS_KEY_ID': 'YOUR_S3_KEY',
    'S3_SECRET_ACCESS_KEY': 'YOUR_S3_SECRET',
    'S3_REGION_NAME': 'region',
    'S3_ENDPOINT_URL': 'YOUR_S3_ENDPOINT',
    'S3_BUCKET_NAME': 'YOUR_BUCKET_NAME',
    
    # LLM endpoint (assumes LLM service deployed in cluster)
    'LLM_ENDPOINT': 'http://llama4-scout-service:8000/v1/chat/completions',
}

# Submit training job
# Ray's working_dir uploads our Zelda code to all workers!
submission_id = client.submit_job(
    entrypoint="python run-ray-zelda.py",
    runtime_env={
        "env_vars": env_vars,
        'working_dir': './',  # Uploads our Zelda code, configs, and ROMs
        'pip': [],  # DD image already has everything!
        "excludes": ["*.sh", "*.ipynb", "*.md", "__pycache__", "*.pyc", "checkpoints/", "training_runs/"],
    }
)

print(f"\n✅ Job submitted successfully!")
print(f"Submission ID: {submission_id}")
print(f"Monitor at: {clusterDetails.dashboard}")


In [ ]:
# Monitor job status
from ray.job_submission import JobStatus

while True:
    status = client.get_job_status(submission_id)
    print(f"Job status: {status}")
    
    if status in [JobStatus.SUCCEEDED, JobStatus.FAILED, JobStatus.STOPPED]:
        print(f"\n📊 Final job logs:\n{client.get_job_logs(submission_id)}")
        break
    
    time.sleep(30)  # Check every 30 seconds


In [ ]:
# Get job logs (anytime)
logs = client.get_job_logs(submission_id)
print(logs)


In [ ]:
# Clean up (when training is complete)
# cluster.down()
